# Lab 2 - Pathway analysis and Machine Learning



# Pathway analysis

Let's pick up from where we left in the last lab. By the end of the lab you found interesting genes that were **differentially expressed** between two **clinically relevant** conditions.

(Note: the selection of relevant clinical conditions is even more important in this lab, if you haven't spent some time on it previously, take the time now.)

You've also learned that one way to make more sense of these results is by performing **pathway analysis**, so let's try that.

We, as always, start by importing relevant libraries and loading the data.
For pathway analysis, we will be mostly working with the [gseapy](https://github.com/ostrokach/gseapy) library, which is mostly a python wrapper for GSEA and Enrichr.


In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gseapy as gp
from gseapy.plot import gseaplot
import qvalue

from ipywidgets import interact, interact_manual
from ipywidgets import IntSlider, FloatSlider, Dropdown, Text

import sklearn as skl
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA


interact_enrich=interact_manual.options(manual_name="Enrichment analysis")
interact_plot=interact_manual.options(manual_name="Plot")
interact_calc=interact_manual.options(manual_name="Calculate tests")

interact_gen=interact_manual.options(manual_name="Initialize data")
interact_SVM=interact_manual.options(manual_name="Train SVM")

clinical_data = pd.read_csv('../data/brca_clin.tsv.gz', sep ='\t', index_col=2)
clinical_data = clinical_data.iloc[4:,1:]
expression_data = pd.read_csv('../data/brca.tsv.gz', sep ='\t', index_col=1)
expression_data = expression_data.iloc[:,2:].T

## 1 Over Representation Analysis

We beggin with Enrichr, which performs **Over Representation Analysis (ORA)**.
You have learned that ORA compares two set of genes and calculates **how likely would their overlap occur by random**.

We will call the first set of genes the 'query set', and they will be the genes found to be *most significantly differentially expressed*, but you will get a chance to define what that means.
Here, you can separate the query genes from the rest by thresholding on either the *p-value* or *q-value*, and the log2 fold change.

The second gene set on the test is the **pathway**, and you will be able to select one of many pathway databases available online. Each database divides the genome into sets according to different criteria.

The test will then compare the overlap between your query set of genes and all the pathways on the database, returning the corresponding statistics.

Interactive fields:
* **Pathway_DB**: Your choice of pathway database.
* **Statistic**: Which statistic to use for thresholding, p or q-value.
* **Threshold**: The statistical threshold.
* **Lo2FC_threshold**: The log2 fold change threshold.

Use the interactive app below to answer the following questions:

### Questions

* 1.1 Imagine an organism that has 10 genes, and a pathway that is composed of all those genes. You run an experiment, test all 10 genes, and find 3 genes that are differentially expressed and belong to that pathway. What would be the p-value for enrichment on that pathway according to ORA?
* 1.2 What difference does it make to threshold the list by p-value or q-value? (Hint: think of the FDR of the resulting enriched pathways)
* 1.3 Did you choose to threshold by the log2 fold change? Why?
* 1.4 Did you relate your choice of thresholds to the nature of the clinical groups you are studying?
* 1.5 After finding good parameters, give a brief interpretation of the pathways that are significantly enriched for this condition.



In [ ]:
def ttest_col(col):
    fc = np.mean(col[index1]) - np.mean(col[index2])
    p_val = sp.stats.ttest_ind(col[index1], col[index2]).pvalue
    return [p_val, fc]

def differential_test(clinical_df, expression_df, separator, cond1, cond2):
    global index1, index2
    try:
        group1 = clinical_df[separator] == cond1
        index1 = clinical_df[group1].index
        group2 = clinical_df[separator] == cond2
        index2 = clinical_df[group2].index
    except:
        print('Clinical condition wrong')
        
    results = expression_df.apply(ttest_col, axis=0, result_type="expand")
    results = results.T
    results.columns = ['p', 'log2fc']
    
    return results

def differential_test_slow(clinical_df, expression_df, separator, cond1, cond2):
    results = pd.DataFrame(columns = ['p','log2fc'])
    try:
        group1 = clinical_df[separator] == cond1
        index1 = clinical_df[group1].index
        group2 = clinical_df[separator] == cond2
        index2 = clinical_df[group2].index
    except:
        print('Clinical condition wrong')
        
    expression1 = expression_df.loc[index1]
    expression2 = expression_df.loc[index2]
    
    for gene in expression_df.columns:
        p_val = sp.stats.ttest_ind(expression1[gene], expression2[gene]).pvalue
        fc = np.mean(expression1[gene]) - np.mean(expression2[gene])
        if p_val == p_val:
            results.loc[gene,'p'] = p_val
            results.loc[gene, 'log2fc'] = fc
            
    return results

def plot_hist(stats, bins):
    stats = np.array(stats)
    plt.hist(stats, bins = bins)
    plt.show()


def interact_multiple_gene_ttest(Criteria, Group_1, Group_2):
    global BRCA_tests
    BRCA_tests = differential_test(clinical_data, expression_data, Criteria, Group_1, Group_2)
    BRCA_tests = qvalue.qvalues(BRCA_tests)
    plot_hist(BRCA_tests['p'].values, 20)
    with pd.option_context('display.max_rows', None):
        display(BRCA_tests)
        
def ORA(tests, threshold, log2fc_threshold, pathway_db=['KEGG_2019_Human'], stat = 'p'):
    background=set(tests.index)
    #gene_list = list(tests.loc[tests[stat]<threshold,stat].index)
    gene_list = list(tests.loc[(tests[stat]<threshold) & (np.abs(tests['log2fc']) > log2fc_threshold), stat].index)
    print('Query set size: ' + str(len(gene_list)))

    output_enrichr=pd.DataFrame()
    enr=gp.enrichr(
                    gene_list=gene_list,
                    gene_sets=pathway_db,
                    background=background,
                    outdir = None
                )
    results = enr.results[["P-value","Overlap","Term"]].rename(columns={"P-value": "p"})
    return qvalue.qvalues(results)

pathway_db_choice = gp.get_library_name()

        
def interact_ORA(Pathway_DB, Statistic, Threshold, Log2FC_threshold):
    threshold = float(Threshold)
    log2fc_threshold = float(Log2FC_threshold)
    results = ORA(BRCA_tests, threshold, log2fc_threshold, Pathway_DB, stat = Statistic)
    with pd.option_context('display.max_rows', None):
        display(results)
        
interact_calc(interact_multiple_gene_ttest, Criteria=Text('Surgical procedure first'), Group_1 = Text('Simple Mastectomy'), Group_2=Text('Lumpectomy'))
interact_enrich(interact_ORA, Threshold = '5e-2' , Pathway_DB = pathway_db_choice, Statistic=['p','q'], Log2FC_threshold = Text('0'))

## 2 Functional Class Scoring

We then move on to another form of pathway analysis, dubbed **Functional Class Scoring**.
As opposed to ORA, this type of enrichment analysis takes as an input a ranked list and a gene set, and asks **how likely is that the genes from that set are randomly distributed along the list**.

Then there is no need to define a query set (!). We already have our list of genes, but we have to find a relevant way to rank it.

Below you will have the option of ranking the list by either p-value, q-value or the log2 fold change.

Perform the enrichment analysis as many times as you think it is needed to answer the question. The test might take a while, so be patient and make sure you have the right parameters before proceeding :)

New interactive fields:
* **Ranking**: which metric to use when ranking the results.

### Questions

* 2.1 Imagine now that you have list of 100 genes ranked by the differential expression q-value, and you have a pathway of only 1 gene. What would be the p-value for enrichment for this pathway under GSEA?
* 2.2 Here, what is the difference between ranking genes by p-value and q-value here?
* 2.3 What are the advantages and disadvantages of ranking by log2 fold change instead (of p/q-value)?
* 2.4 Can you think of a better way or ranking the analytes than the ones you were presented with? Cite one advantage and one disadvantage of your proposed ranking method.
* 2.5 Again, what is your interpretation of the pathways that are significantly enriched for this clinical conditions? Did you find any difference from the ones found performing ORA? If you found any difference, why do you think these pathways are found to be enriched in one case and not the other?

In [ ]:
def gsea(tests, stat, pathway_db = 'KEGG_2019_Human'):
    pre_res = gp.prerank(rnk=tests[stat], 
                    gene_sets=pathway_db,
                    processes=4,
                    permutation_num=100, # reduce number to speed up testing
                    outdir=None, format='png')
    return pre_res

def interact_gsea(Pathway_DB, Ranking):
    global Results_gsea
    Results_gsea = gsea(BRCA_tests.dropna(), pathway_db=Pathway_DB, stat = Ranking)
    with pd.option_context('display.max_rows', None):
        display(Results_gsea.res2d[['pval', 'fdr']])
        
interact_calc(interact_multiple_gene_ttest, Criteria=Text('Surgical procedure first'), Group_1 = Text('Simple Mastectomy'), Group_2=Text('Lumpectomy'))
interact_enrich(interact_gsea, Pathway_DB = pathway_db_choice, Ranking = ['p','q','log2fc'])

# Machine learning
# 3 Support Vector Machine

Ok, so now for something different.
We continue with our dataset on breast cancer, but this time we will try to use machine learning to **predict the clinical conditions based on the gene expression data**.

You again will be able two choose the two clinical groups we will use to train the model, and your choice will affect the results and performance of the model. Special attention should be paid to the number of data points in each group, which will be displayed after you select the groups.

Let's train our first SVM. You've learned the merits of feature scaling (or in other terms, normalizing the input), and so we will test it.
Below you will separate the data set into two, and train a SVM to distinguish between the two using the gene expression values of the samples. You can choose whether you'd like to rescale (normalize) the features first or not.

Once you have done so, you will be presented with a [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix), and from that will need to devise a performance metric.
[This Wikipedia page](https://en.wikipedia.org/wiki/Evaluation_of_binary_classifiers) can be helpful when choosing how to measure the performance. You might want to do that before running the SVM tests.

You know the drill by now, play around with it for a while and answer the questions.

Interactive fields:
* **Rescale**: whether to apply mean and variance normalization before training the SVM.
* **Data_split**: the fraction of the data that is left on the test set.
* **Max_iterations**: the maximum number of iterations the SVM is allowed to perform, increase it if you are seeing convergence errors.

## Questions

* 3.1 Report the size of your two groups. Which performance metric did you choose, and how did you take the data into account on this decision? Did you choose this metric before running the tests or after?
* 3.2 How much did you chose to leave out for the test set? How did this affect your performance and your ability to measure that performance?
* 3.3 Did you achieve better performance by normalizing the gene expression? If you did, what new information do you think the normalization adds to the model?
* 3.4 From the clinical groups you've tested, can you tell any pattern on which ones are better suited for this type of classification?







In [ ]:
def split_data(clinical_df, expression_df, separator, cond1, cond2):
    try:
        group1 = clinical_df[separator] == cond1
        index1 = clinical_df[group1].index
        group2 = clinical_df[separator] == cond2
        index2 = clinical_df[group2].index
    except:
        print('Clinical condition wrong')
    expression1 = expression_df.loc[index1].dropna()
    expression2 = expression_df.loc[index2].dropna()
    expression = pd.concat([expression1, expression2])
    X = expression.values
    y = np.append(np.repeat(0, len(expression1)), np.repeat(1, len(expression2)))
    display(pd.DataFrame([len(index1),len(index2)], columns = ['Number of points'], index = ['Group 1', 'Group 2']))
    return X, y

def train_SVM(X, y, C=1, scale = False, max_iter = 1000):
    if scale:
        scaler = StandardScaler()
        X = scaler.fit_transform(X)
    clf = LinearSVC(C=C, max_iter=max_iter)
    clf.fit(X,y)
    return clf

def print_accuracy(X_train, y_train, X_test, y_test, clf):
    y_train_pred = clf.predict(X_train)
    ac_matrix_train = confusion_matrix(y_train, y_train_pred)
    y_test_pred = clf.predict(X_test)
    ac_matrix_test = confusion_matrix(y_test, y_test_pred)
    display(pd.DataFrame(np.concatenate((ac_matrix_train,ac_matrix_test), axis =1), columns = ["predicted G1 (training)","predicted G2 (training)", "predicted G1 (test)","predicted G2 (test)"],index=["actual G1","actual G2"]))
    
def plot_pca_variance(X, scale=False, ncomp = 1):
    if scale:
        scaler = StandardScaler()
        X = scaler.fit_transform(X)
    pca = PCA()
    pca.fit(X)
    plt.rcParams["figure.figsize"] = (20,10)
    sns.set(style='darkgrid', context='talk')
    plt.plot(np.arange(1,len(pca.explained_variance_ratio_)+1),np.cumsum(pca.explained_variance_ratio_))
    plt.xlabel('Number of components')
    plt.ylabel('Cumulative explained variance')
    
    plt.vlines(ncomp, 0, plt.gca().get_ylim()[1], color='r', linestyles = 'dashed')
    h = np.cumsum(pca.explained_variance_ratio_)[ncomp -1]
    plt.hlines(h, 0, plt.gca().get_xlim()[1], color='r', linestyles = 'dashed')
    plt.title(str(ncomp) + ' components, ' + str(round(h, 3)) + ' variance explained')
    plt.show()
    
def reduce_data(X, n, scale=True):
    if scale:
        scaler = StandardScaler()
        X = scaler.fit_transform(X)
    pca = PCA(n_components=n)
    Xr = pca.fit_transform(X)
    return Xr

def interact_split_data(Criteria, Group_1, Group_2):
    global BRCA_X, BRCA_y
    BRCA_X, BRCA_y = split_data(clinical_data, expression_data, Criteria, Group_1, Group_2)

def interact_SVM_1(Rescale, Data_split, Max_iterations):
    max_iter = int(Max_iterations)
    X_train, X_test, y_train, y_test = train_test_split(BRCA_X, BRCA_y, test_size=Data_split)
    clf = train_SVM(X_train, y_train, C=1, scale = Rescale, max_iter=max_iter)
    print_accuracy(X_train, y_train, X_test, y_test, clf)
    
interact_gen(interact_split_data, Criteria=Text('Surgical procedure first'), Group_1 = Text('Simple Mastectomy'), Group_2=Text('Lumpectomy'))
interact_SVM(interact_SVM_1, Rescale = False, Data_split = FloatSlider(min=0,max=1,value=0.1, step = 0.05), Max_iterations = Text('1000'))

# 4 Regularization

You have learned about the propensity of machine learning methods to overfit.
One way of trying to avoid overfitting is called [**regularization**](https://en.wikipedia.org/wiki/Regularization_(mathematics)).
Regularization, as the names suggests, is a way of adding extra rules in a way that adds extra information on the model. It also serves as a great way to add **domain knowledge** into a general model.

SVMs have a built in regularization mechanism in the form of the **C parameter**. If you are not familiar with this parameter, read a bit on it, and then play around with the SVM below.
As with every form of regularization, it is more of an art than a science, and so a good value will depend a lot on how your data looks like.

New interactive fields:
* **C_parameter**: the value of the C parameter.

## Questions

* 4.1 Explain in your own words what the parameter C is doing, and why do you think it can be considered a form or regularization.
* 4.2 Explain how (or if) do you think it is possible to "add expert knowledge" to the system by simply modifying the C parameter?
* 4.3 Did you manage to find an good value (or range) for C that improves performance? Does it change between different clinical groups?
* 4.4 Do you think it is better to set a value of C based on the **best performance** of the classifier, or based on **prior information** about the data?

In [ ]:
def interact_SVM_2(Rescale, Data_split, Max_iterations, C_parameter):
    max_iter = int(Max_iterations)
    C = float(C_parameter)
    X_train, X_test, y_train, y_test = train_test_split(BRCA_X, BRCA_y, test_size=Data_split)
    clf = train_SVM(X_train, y_train, C=C, scale = Rescale, max_iter=max_iter)
    print_accuracy(X_train, y_train, X_test, y_test, clf)

interact_gen(interact_split_data, Criteria=Text('Surgical procedure first'), Group_1 = Text('Simple Mastectomy'), Group_2=Text('Lumpectomy'))
interact_SVM(interact_SVM_2, Rescale = False, Data_split = FloatSlider(min=0,max=1,value=0.1, step = 0.05), Max_iterations = Text('1000'), C_parameter = Text('1'))

# 5 Dimensionality

Let's explore how the dimensionality of the data affects the prediction accuracy. You have heard of the [curse of dimensionality](https://en.wikipedia.org/wiki/Curse_of_dimensionality) and, in bioinformatics, often we are subject to it just because of the amount of genes on a genome and the low number of samples.

Luckily, you have also learned at least one way to reduce the dimensionality of the data. Here we will use PCA to reduce the dimensions from around 20 thousand genes (dimensions) to a more manageable number of dimensions.
First, you will need to choose if you normalize all the dimensions before performing the PCA, then you will be shown how much of the variance in the data is captured by the first components (remember that the normalization also changes the variance). With this information you will be able to have an educated guess on how many principal components you want to use to train the SVM.

As always, play with it as much as you need to answer the question.

New interactive fields:

* **PCA_scaling**: whether to apply the normalization before performing the PCA.
* **N_components**: number of principal components selected.
* **Reduce_dim**: whether to reduce the dimentions of the data before training the SVM.

## Questions

* 5.1 Explain using your own words the **curse of dimensionality**. Why adding more information doesn't always increase the performance?
* 5.2 Do you think the features should be normalized before performing the PCA? Why?
* 5.3 How many principal components did you chose at first, and how much of the variance did they capture? Why? Did it turn out to be the optimum number of dimensions or was it too high/low?
* 5.4 In the end, did the dimensionality reduction help the classifier? Why do you think that this works/doesn't in this case?

In [ ]:
def interact_PCA_plot(PCA_scaling, N_components):
    n_comp = int(N_components)
    plot_pca_variance(BRCA_X, scale=PCA_scaling, ncomp = n_comp)
    
def interact_SVM_3(Rescale, Data_split, Max_iterations, C_parameter, Reduce_dim, PCA_scaling, N_components):
    max_iter = int(Max_iterations)
    n_comp = int(N_components)
    C = float(C_parameter)
    if Reduce_dim:
        X = reduce_data(BRCA_X, n = n_comp, scale=PCA_scaling)
    else:
        X = BRCA_X
    X_train, X_test, y_train, y_test = train_test_split(X, BRCA_y, test_size=Data_split)
    clf = train_SVM(X_train, y_train, C=C, scale = Rescale, max_iter=max_iter)
    print_accuracy(X_train, y_train, X_test, y_test, clf)
    
interact_gen(interact_split_data, Criteria=Text('Surgical procedure first'), Group_1 = Text('Simple Mastectomy'), Group_2=Text('Lumpectomy'))
interact_plot(interact_PCA_plot, PCA_scaling = False, N_components = Text('1'))
interact_SVM(interact_SVM_3, Rescale = False, Data_split = FloatSlider(min=0,max=1,value=0.1, step = 0.05), Max_iterations = Text('1000'), C_parameter = Text('1'), Reduce_dim = False, PCA_scaling = False, N_components = Text('1'))

# Bonus

**Note**: To get full points in the **bonus part** you only have to answer 3 out of the 4 questions below, but you **have to specify** on your report which question you are leaving out.

## ORA again
* B1.1 Imagine again that organism with 10 genes (question 1.1), but now the pathway has only 5 genes, and all 3 differentially expressed genes belong to it. What is the p-value for the enrichment of that pathway? Show how you arrived at the result. If you use the hypergeometric test, explain what all the terms mean. (Hint: you may want to use the [binomial coefficient](https://en.wikipedia.org/wiki/Combination) to help you calculate)

## Bias-variance trade-off

Bias and variance are both sources of error in any predictive model. The bias-variance trade-off principle states that usually models with high bias error have low variance error, and vice-versa.

* B2.1 Read more about **bias** and **variance** errors and explain with your own words what each of them mean.
* B2.2 Explain with your own words the trade-off, why it occurs, and how the error is decomposed in this two terms. Use figures if think it's useful.
* B2.3 How does this relate to model complexity and overfitting?
* B2.4 How would you control the trade-off in SVMs?
* B2.5 How would you control it in clustering?

## Cross validation

Some times you don't have enough data, and it is a shame to leave some of it out of the training set. One way to mitigate this is by using cross validation.

* B3.1 Implement cross validation in one of the SVMs you used above, explaining the decisions you made during the implementation, i.e. in how many parts you've divided your set, etc.
* B3.2 Also implement "leave one out" cross validation
* B3.3 Compare the results above with the results from previous classifiers in this lab. Do you achieve better performance?

## Parameter optimization

For the questions below, we will explore more in depth how some parameter affect the performance by doing [parameter optimization](https://en.wikipedia.org/wiki/Hyperparameter_optimization).

Perform a parameter optimization, using cross validation, for the following parameter on an SVM you used above. You can use ready functions such as [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) to help. Plot and interpret the results.

* B4.1 Number of dimensions of the input (principal components)
* B4.2 The C parameter
* B4.3 The C parameter and number of dimensions, and scaling (of the input) at the same time.
* B4.4 Comment about the relationship between the variables by using the results above.
* B4.5 Now, investigate (briefly) the claim that cross validation shouldn't be use to report performance. You may want to use [this paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1397873/) to do so. Give your opinion on the matter.

